In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn. metrics import roc_auc_score, accuracy_score
import seaborn as sns

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Deep_learning_data/Riiid_data/train_1M_with_features_merged_v1.csv')

FileNotFoundError: [Errno 2] File b'/content/drive/MyDrive/Deep_learning_data/Riiid_data/train_1M_with_features_merged_v1.csv' does not exist: b'/content/drive/MyDrive/Deep_learning_data/Riiid_data/train_1M_with_features_merged_v1.csv'

In [ ]:
from sklearn.preprocessing import RobustScaler
corr = df.query('content_type_id==0').corr().answered_correctly

In [ ]:
#corr[corr > 0.04].sort_values(ascending=False).index.to_list()
features = ['user_personalized_qstat_knowing_had_explanation_or_not',
 'qstats_answered_correctly',
 'user_avg_score_cum',
 'qstats_prior_question_had_explanation',
 'already_seen',
 'already_seen_tag_lecture',
 'user_activity_cumcount',
 'qstats_user_activity_cumcount',
 'at_least_one_lesson',
 'qstats_user_activity_cumcount_knowing_answered_correctly',
 'qstats_task_container_id_knowing_answered_correctly',
 'lectures_seen',
 'qstats_count',
 'qstats_answers',
 'content_type_id']

In [ ]:
import numpy as np
features = df[features].replace(np.inf, np.nan)

In [ ]:
features.replace(np.nan, 0, inplace=True)

Best so far - XGBoost


In [ ]:
#scaler.fit_transform(X_train, y_train)
features.query('content_type_id==0').drop(columns={'content_type_id'})

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

X = features.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 2)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

xgboost = GradientBoostingClassifier()
xgboost.fit(X_train, y_train)

y_pred = cross_val_predict(xgboost, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7681833689157577


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  9.0min finished


In [ ]:
xgboost.feature_importances_

array([7.53592596e-01, 6.86981811e-03, 1.46317284e-01, 4.22763793e-03,
       5.97288637e-02, 4.05674787e-04, 1.70034080e-02, 6.10444112e-04,
       1.21433645e-03, 4.21925696e-03, 7.44036813e-04, 4.47050451e-03,
       5.96138663e-04, 0.00000000e+00])

In [ ]:
features.columns

Index(['user_personalized_qstat_knowing_had_explanation_or_not',
       'qstats_answered_correctly', 'user_avg_score_cum',
       'qstats_prior_question_had_explanation', 'already_seen',
       'already_seen_tag_lecture', 'user_activity_cumcount',
       'qstats_user_activity_cumcount', 'at_least_one_lesson',
       'qstats_user_activity_cumcount_knowing_answered_correctly',
       'qstats_task_container_id_knowing_answered_correctly', 'lectures_seen',
       'qstats_count', 'qstats_answers', 'content_type_id'],
      dtype='object')

In [ ]:
joblib.dump(xgboost, "~/code/YohannD/riiid-project/heroku/data)")

In [2]:
~/code/YohannD

SyntaxError: invalid syntax (<ipython-input-2-6772fd1077df>, line 1)

Logistic Regression

In [ ]:
X = df.query('content_type_id==0')[['qstats_answered_correctly', 'user_avg_score_cum', 'already_seen', 'lectures_seen', 'at_least_one_lesson']]
y = df.query('content_type_id==0')['answered_correctly']

In [ ]:
X = features.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 2)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

log = LogisticRegression()
log.fit(X_train, y_train)
y_pred = cross_val_predict(log, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

Correlations

In [ ]:
corr.sort_values(ascending=False)

In [ ]:
corr = df.query('content_type_id==0').corr().answered_correctly

RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = df.query('content_type_id==0')[['qstats_answered_correctly', 'user_avg_score_cum', 'already_seen', 'lectures_seen', 'at_least_one_lesson']]
y = df.query('content_type_id==0')['answered_correctly']

In [ ]:
y_pred = cross_val_predict(forest, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7125921037505767


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  7.0min finished


Bagging Classifier

Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 1)
adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=50) 
adaboost.fit(X_train, y_train)

y_pred = cross_val_predict(adaboost, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7594259462782705


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.3min finished


SVC

In [ ]:
from sklearn.svm import SVC

X = features.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']

for kernel in kernels:
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 2)
  X_train = scaler.fit_transform(X_train)
  svc = SVC(kernel=kernel, C=10, max_iter=1500, probability=True, tol=0.01, gamma='scale')
  svc.fit(X_train, y_train)
  y_pred = cross_val_predict(svc, X_test, y_test, cv=5, method='predict_proba', verbose=1)
  print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

In [ ]:
# equivalent but with SGD solver
from sklearn.linear_model import SGDClassifier

svc_bis = SGDClassifier(loss='modified_huber', penalty='l2', alpha=0.1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 1)

#svc_bis = GradientBoostingClassifier() 
svc_bis.fit(X_train, y_train)

y_pred = cross_val_predict(svc_bis, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7289342611288083


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    4.3s finished


Multi-layer Stacking - XGBoost, SVC and Log reg

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

X = features.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 2)

X_train = scaler.fit_transform(X_train)


ensemble = StackingClassifier(
    estimators = [("log", GradientBoostingClassifier()),
                  ("svc", SGDClassifier(loss='modified_huber', penalty='l2', alpha=0.1, max_iter=3000))],
                  final_estimator = LogisticRegression(max_iter=3000) )

ensemble.fit(X_train, y_train)
y_pred = cross_val_predict(ensemble, X_test, y_test, cv=5, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7682322465652832


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 25.1min finished


In [4]:
df = pd.read_csv('../data/train_1M.csv')
df.head()

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,90286654,90286654,0,1917429896,7900,0,0,0,1,NaN,NaN
1,90286655,90286655,26511,1917429896,7876,0,1,2,0,21000.0,False
2,90286656,90286656,55312,1917429896,175,0,2,0,0,21000.0,False
3,90286657,90286657,102436,1917429896,1278,0,3,0,0,23000.0,False
4,90286658,90286658,188454,1917429896,2063,0,4,3,0,45000.0,False


In [15]:
long_user = df.query('user_id==1641944791')

In [17]:
long_user.to_csv('longuser_test.csv', index=False)

In [18]:
pwd

'/Users/Yohann/code/YohannD/riiid-project/notebooks'